In [1]:
#!/usr/bin/python
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import os
import glob
# from model import *
from functions import *
import pdb
import scipy.stats as stats
from scipy.signal import spectrogram

# Go to the directory and print the *.csv files 

In [2]:
directory = '/raid/togzhan_syrymova/lump_project/data/'
os.chdir(directory)
# glob.glob('*.csv')

# Load the datasets 

In [3]:
directory = '/raid/togzhan_syrymova/lump_project/data/'
os.chdir(directory)
csv_files_train = glob.glob('df_train*raw*8.csv')
csv_files_test = glob.glob('test*raw*.csv')
csv_files_dev = glob.glob('df_dev*raw*8.csv')

index = 0

df_train_ = pd.read_csv(csv_files_train[index], sep = ',', header = None)
df_train_ = pd.DataFrame(df_train_.values[1:,:])

df_test1 = pd.read_csv(csv_files_test[index], sep = ',', header = None)
df_test2 = pd.read_csv(csv_files_test[index+1], sep = ',', header = None)
df_test_ = pd.concat([df_test1, df_test2], axis = 1)
df_test_ = df_test_.transpose()

df_dev_ = pd.read_csv(csv_files_dev[index], sep = ',', header = None)
df_dev_ = pd.DataFrame(df_dev_.values[1:,:])


print(csv_files_train, '\n')
print(csv_files_test, '\n')
print(csv_files_dev, '\n')

print(csv_files_train[index], '\n')
print(csv_files_test[index], '\n')
print(csv_files_dev[index], '\n')

['df_train_raw_8.csv'] 

['test_pressure_data_without_raw.csv', 'test_pressure_data_with_raw.csv'] 

['df_dev_raw_8.csv'] 

df_train_raw_8.csv 

test_pressure_data_without_raw.csv 

df_dev_raw_8.csv 



# Get labels and data separately

In [4]:
# Train and dev labels
train_y11 = pd.DataFrame((df_train_.values[:,-1].astype('float64')))
dev_y11 = pd.DataFrame((df_dev_.values[:,-1].astype('float64')))

# Create test labels
test_y11 = pd.concat([ pd.DataFrame([0 for i in range(2*32*9)]), pd.DataFrame([1 for i in range(2*32*9)])], axis=0)
test_y11 = test_y11.reset_index(drop=True)


df_train1 = pd.DataFrame(df_train_.values[:,:-1].astype('float64'))
df_dev1 = pd.DataFrame(df_dev_.values[:,:-1].astype('float64'))
df_test1 = df_test_.astype('float64')

## SHUFFLE the TEST labels and data
test_y1, df_test1 = shuffle(test_y11.astype('float64'), df_test1)

# Compute mean and stdev of the whole sequence per sensor for train/dev/test sets

In [5]:
# idx = 0
# df_train, df_dev, df_test = get_data(idx, df_train1, df_dev1, df_test1)

# Compute mean and stdev of the 1 sec of sequence per sensor for train/dev/test sets

In [6]:
# idx = 1
# df_train, train_y2, df_dev,dev_y2, df_test, test_y2 = get_data(idx, df_train1, df_dev1, df_test1)

# Run to take imported sequence with no changes

In [7]:
# import pandas as pd
# import functions
# idx = 2
# df_train, df_dev, df_test = get_data(idx, df_train1, df_dev1, df_test1)

In [8]:
def get_data(idx, df_train1, df_dev1, df_test1):
    df_train_mean = pd.DataFrame()
    df_train_stdev = pd.DataFrame()

    df_test_mean = pd.DataFrame()
    df_test_stdev = pd.DataFrame()

    df_dev_mean = pd.DataFrame()
    df_dev_stdev = pd.DataFrame()
    if (idx == 0):
        for i in range(15):
            df_train_mean = pd.concat([df_train_mean, pd.DataFrame(np.mean(df_train1.values[:, i*1120:(i+1)*1120], axis = 1))], axis = 1)
            df_train_stdev = pd.concat([df_train_stdev, pd.DataFrame(np.std(df_train1.values[:, i*1120:(i+1)*1120], axis = 1))], axis = 1)

            df_dev_mean = pd.concat([df_dev_mean, pd.DataFrame(np.mean(df_dev1.values[:, i*1120:(i+1)*1120], axis = 1))], axis = 1)
            df_dev_stdev = pd.concat([df_dev_stdev, pd.DataFrame(np.std(df_dev1.values[:, i*1120:(i+1)*1120], axis = 1))], axis = 1)

            df_test_mean = pd.concat([df_test_mean, pd.DataFrame(np.mean(df_test1.values[:, i*1120:(i+1)*1120], axis = 1))], axis = 1)
            df_test_stdev = pd.concat([df_test_stdev, pd.DataFrame(np.std(df_test1.values[:, i*1120:(i+1)*1120], axis = 1))], axis = 1)

        df_train = pd.concat([df_train_mean, df_train_stdev], axis = 1)
        df_dev = pd.concat([df_dev_mean, df_dev_stdev], axis = 1)
        df_test = pd.concat([df_test_mean, df_test_stdev], axis = 1)
        return df_train, df_dev, df_test
    
    elif (idx == 1):
        train_y2 = pd.DataFrame()
        dev_y2 = pd.DataFrame()
        test_y2 = pd.DataFrame()

        for j in range(7):
            df_train_mean_s = pd.DataFrame()
            df_train_stdev_s = pd.DataFrame()

            df_test_mean_s = pd.DataFrame()
            df_test_stdev_s = pd.DataFrame()

            df_dev_mean_s = pd.DataFrame()
            df_dev_stdev_s = pd.DataFrame()

            for i in range(15):
                index1 = i*1120+j*160
                index2 = i*1120+(j+1)*160
                df_train_mean_s = pd.concat([df_train_mean_s, pd.DataFrame(np.mean(df_train1.values[:, index1:index2], axis = 1))], axis = 1)
                df_train_stdev_s = pd.concat([df_train_stdev_s, pd.DataFrame(np.std(df_train1.values[:, index1:index2], axis = 1))], axis = 1)

                df_dev_mean_s = pd.concat([df_dev_mean_s, pd.DataFrame(np.mean(df_dev1.values[:, index1:index2], axis = 1))], axis = 1)
                df_dev_stdev_s = pd.concat([df_dev_stdev_s, pd.DataFrame(np.std(df_dev1.values[:, index1:index2], axis = 1))], axis = 1)

                df_test_mean_s = pd.concat([df_test_mean_s, pd.DataFrame(np.mean(df_test1.values[:, index1:index2], axis = 1))], axis = 1)
                df_test_stdev_s = pd.concat([df_test_stdev_s, pd.DataFrame(np.std(df_test1.values[:, index1:index2], axis = 1))], axis = 1)

            df_train_mean = pd.concat([df_train_mean, df_train_mean_s], axis = 0)
            df_train_stdev = pd.concat([df_train_stdev, df_train_stdev_s], axis = 0)

            df_test_mean = pd.concat([df_test_mean, df_test_mean_s], axis = 0)
            df_test_stdev = pd.concat([df_test_stdev, df_test_stdev_s], axis = 0)

            df_dev_mean = pd.concat([df_dev_mean, df_dev_mean_s], axis = 0)
            df_dev_stdev = pd.concat([df_dev_stdev, df_dev_stdev_s], axis = 0)

            # concat labels 7 times corresponding to 7 second
            train_y2 = pd.concat([train_y2, train_y11], axis = 0)
            dev_y2 = pd.concat([dev_y2, dev_y11], axis = 0)
            test_y2 = pd.concat([test_y2, test_y11], axis = 0)


        df_train = pd.concat([df_train_mean, df_train_stdev], axis = 1)
        df_dev = pd.concat([df_dev_mean, df_dev_stdev], axis = 1)
        df_test = pd.concat([df_test_mean, df_test_stdev], axis = 1)
        
    elif(idx == 2):
        df_train = df_train1
        df_dev = df_dev1
        df_test = df_test1
        return df_train, df_dev, df_test
    
    elif(idx == 3): #COMPUTE z-score
        df_train_z = pd.DataFrame()
        df_test_z = pd.DataFrame()
        df_dev_z = pd.DataFrame()
        for i in range(15):
            df_train_z1 = pd.DataFrame()
            df_dev_z1 = pd.DataFrame()
            df_test_z1 = pd.DataFrame()

            index1 = i*1120
            index2 = (i+1)*1120

            d_train = df_train1.values[:, i*1120:(i+1)*1120]
            d_dev = df_dev1.values[:, i*1120:(i+1)*1120]
            d_test = df_test1.values[:, i*1120:(i+1)*1120] 

            df_train_z1 = pd.DataFrame(stats.zscore(d_train, axis = 1))
            df_dev_z1 = pd.DataFrame(stats.zscore(d_dev, axis = 1))
            df_test_z1 = pd.DataFrame(stats.zscore(d_test, axis = 1))

            df_train_z1.columns = [''] * len(df_train_z1.columns)
            df_dev_z1.columns = [''] * len(df_dev_z1.columns)
            df_test_z1.columns = [''] * len(df_test_z1.columns)

        #     df_train_z1 = df_train_z1.reset_indext_index()
            df_train_z = pd.concat([df_train_z, df_train_z1], axis = 1)
            df_dev_z = pd.concat([df_dev_z, df_dev_z1], axis = 1)
            df_test_z = pd.concat([df_test_z, df_test_z1], axis = 1)
            
        return df_train_z, df_dev_z, df_test_z
    
    elif(idx == 4): #COMPUTE z-score
        df_train_z = pd.DataFrame()
        df_test_z = pd.DataFrame()
        df_dev_z = pd.DataFrame()
        df_train_z = pd.DataFrame(stats.zscore(df_train1, axis = 1))
        df_dev_z = pd.DataFrame(stats.zscore(df_dev1, axis = 1))
        df_test_z = pd.DataFrame(stats.zscore(df_test1, axis = 1))
            
        return df_train_z, df_dev_z, df_test_z
    
def compute_spectogram(df):
    df_1 = np.zeros([np.shape(df)[0], 129, 4])
    for i in range(15):
        fs = 160
        f, t, data = spectrogram(df.values[:, 1120*i:(i+1)*1120], fs, axis = 1)
        df_1 = np.concatenate([df_1, 10*np.log10(data)], axis = 1)
        data = []
    return df_1[:, 129:, :],t, f

# Compute z-score

In [9]:
# run FUNCTIONS at the end of the notebook
idx = 3
df_train11, df_dev11, df_test11 = get_data(idx, df_train1, df_dev1, df_test1)
train_y1 = train_y11
dev_y1 = dev_y11
test_y1 = test_y1

# Compute z-score per trial

In [10]:
# # run FUNCTIONS at the end of the notebook
# idx = 4
# df_train11, df_dev11, df_test11 = get_data(idx, df_train1, df_dev1, df_test1)
# train_y1 = train_y11
# dev_y1 = dev_y11
# test_y1 = test_y11

# Compute spectogram

In [11]:
# from numpy import newaxis

# def compute_spectogram(df):
#     df_1 = np.zeros([np.shape(df)[0], 129, 4, 1])
#     for i in range(15):
#         fs = 160
#         f, t, data = spectrogram(df.values[:, 1120*i:(i+1)*1120], fs, axis = 1)
# #         print(np.shape(data))
#         data1 = data[:, ..., newaxis]
# #         print(np.shape(data1))
# #         d = 10*np.log10(data1)
#         df_1 = np.concatenate([df_1, data1], axis = 3)
#         data = []
# #         print(np.shape(df_1))
#     return df_1[:, :, :, 1:],t, f

# df_train3, t, f = compute_spectogram(df_train1)
# train_y1 = train_y11
# print(np.shape(df_train3))

# df_dev3, t, f = compute_spectogram(df_dev1)
# dev_y1 = dev_y11
# print(np.shape(df_dev3))
# df_test3, t, f = compute_spectogram(df_test1)
# test_y1 = test_y1
# print(np.shape(df_test3))

# # df_train = df_train3
# # df_dev = df_dev3
# # df_test = df_test3

In [12]:
# shape = np.shape(df_train3)
# df_train = np.reshape(df_train3, (shape[0],shape[1]*shape[2], shape[3]))

# shape = np.shape(df_dev3)
# df_dev = np.reshape(df_dev3, (shape[0],shape[1]*shape[2], shape[3]))


# shape = np.shape(df_test3)
# df_test = np.reshape(df_test3, (shape[0],shape[1]*shape[2], shape[3]))

# Create Tensor from labels and data

In [15]:
from numpy import newaxis
sh = np.shape(df_train11)
a = df_train11.values[:]
df_train = df_train11.values[:, newaxis, :]
df_dev = df_dev11.values[:,newaxis, :]
df_test = df_test11.values[:,newaxis, :]
print(np.shape(df_train))

channels_in = 15

df_train = df_train.reshape(np.shape(df_train)[0], channels_in, np.int64(np.shape(df_train)[2]//channels_in))
df_dev = df_dev.reshape(np.shape(df_dev)[0], channels_in, np.int64(np.shape(df_dev)[2]//channels_in))
df_test = df_test.reshape(np.shape(df_test)[0], channels_in, np.int64(np.shape(df_test)[2]//channels_in))
print(np.shape(df_train))
####


# df_train = df_train11.values[:, :, newaxis]
# df_dev = df_dev11.values[:, :,newaxis]
# df_test = df_test11.values[:, :,newaxis]

# df_train = df_train.reshape(np.shape(df_train)[0], np.int64(np.shape(df_train)[1]//channels_in), channels_in)
# df_dev = df_dev.reshape(np.shape(df_dev)[0], np.int64(np.shape(df_dev)[1]//channels_in), channels_in)
# df_test = df_test.reshape(np.shape(df_test)[0], np.int64(np.shape(df_test)[1]//channels_in), channels_in)
# print(np.shape(df_train))


(4032, 1, 16800)
(4032, 15, 1120)


In [16]:
# Create tensor from labels
# Transform labels in the format 1_9 to binary
train_y3 = pd.DataFrame(np.floor(train_y1.values[:].astype('float64')/10))
dev_y3 = pd.DataFrame(np.floor(dev_y1.values[:].astype('float64')/10))

train_y = torch.FloatTensor(train_y3.values[:])
dev_y = torch.FloatTensor(dev_y3.values[:])
test_y = torch.FloatTensor(test_y1.values[:])

train_x	= torch.FloatTensor(df_train.astype('float64'))
dev_x = torch.FloatTensor(df_dev.astype('float64'))
test_x = torch.FloatTensor(df_test.astype('float64'))

In [ ]:
print(np.shape(train_x))
print(np.shape(train_y))

print(np.shape(dev_x))
print(np.shape(dev_y))

print(np.shape(test_x))
print(np.shape(test_y))

In [ ]:
# from torch.nn import Conv2d, functional as F, Linear, MaxPool2d, Module

# class ConvLayerNet4(torch.nn.Module):

#     def __init__(self,  channels_in, channels_out, D_in, H, D_out, device, drop=0.0, kernel_size=1, stride=1):
#         super(ConvLayerNet4, self).__init__()
        
#         self.device = device
        
#         self.convs = nn.ModuleList([
#             nn.Conv2d(1, channels_out, kernel_size=kernel_size, stride=stride)
#         for i in range(15)])
        
#         W_out = ((D_in - kernel_size)//stride + 1)*channels_out
        
#         self.pool = MaxPool2d(kernel_size=2, stride=2, padding=0)
        
#         self.linear1_1 = torch.nn.Linear(H3, 64)
#         self.linear2 = torch.nn.Linear(64, D_out)
        
#         self.fc1 = Linear(18 * 16 * 16, 64)
#         self.fc2 = Linear(64, 1)

#     def forward(self, x):
#         ii = 0
#         h_relu = torch.FloatTensor([]).to(self.device)
#         for layers in self.convs:
#             print(np.shape(torch.unsqueeze(x[:, ii, :, :], 1)))
#             x_data = layers(torch.unsqueeze(x[:, ii, :, :], 1)) # torch.unsqueeze(x[:, ii, :, :], 1
#             print(np.shape(x_data))
#             x_data = self.relu(x_data)
#             b, h, w = x_data.size()
#             x_data = x_data.view(b,-1)
#             h_relu = torch.cat([h_relu, x_data], axis = 1)
#             ii = ii+1
            
#         x = F.relu(self.conv(x))
#         x = self.pool(x)
#         x = x.view(-1, 18 * 16 * 16)
#         x = F.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x

In [ ]:
# print(np.shape(train_x))
# print(D_in)
# W_out = ((D_in - kernel_size)//stride + 1)*channels_out
# print(W_out)
# #         W_out = ((W_out - kernel_size)//stride + 1)*channels_out
# W_out = ((W_out - kernel_size)//stride + 1)*15*channels_out
# W_out = np.int64(W_out) 

# print(W_out)

# Define the hyperparameters of the model

In [17]:
print('Started training')
seed = 777
torch.manual_seed(seed)

#################
# choose the gpu
gpu_id = 5
device = torch.device("cuda:" + str(gpu_id))
# device = torch.device("cpu")
print(device)

#################
# Input and output sizes
D_in = np.shape(train_x)[2]
channels_in = 1
channels_out = 1

D_out = 1

#################
# Choose the hidden layer, drop, batch_size, num_epochs, learning_rate
# H = 512  #hidden layer size

drop = 0.3
H1 = [ 512, 256, 128]#, 2048,1024, 512
N1 = [8, 16, 32, 64]
epochs = 100
learning_rate = 1e-4
kernel_size = 3
stride  = 2

# loss_fn = torch.nn.BCELoss()BCEWithLogitsLoss
# N = batch_size

best_epoch = 0
dev_acc_max=0
epoch_max=0
loss_list     = np.zeros((epochs,))
accuracy_list = np.zeros((epochs,))


Started training
cuda:5


In [ ]:
D_in

In [18]:
class ConvLayerNet3(torch.nn.Module):
    def __init__(self,  channels_in, channels_out, D_in, H, D_out, device, drop=0.0, kernel_size=1, stride=1):
        super(ConvLayerNet3, self).__init__()
        self.device = device
        self.input_size = 1    # 6 input channels
        self.num_outputs = 1   # controls for two motors
#         self.batch = batchSize

        # model
        self.num_layers = 1
        self.hidden_size = 32
#         self.convs = nn.ModuleList([nn.Conv1d(1, channels_out, kernel_size=kernel_size, stride=stride) for i in range(15)])
        self.lstm = nn.ModuleList([
            nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first = True)
         for i in range(15)])
        
        self.linear_1 = nn.Linear(self.hidden_size*15, H)
        self.linear1_1 = nn.Linear(H, 64)
        self.linear_2 = nn.Linear(64, self.num_outputs)
#         self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, x):
        
        ii = 0
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)
        h_relu = torch.FloatTensor([]).to(self.device)
        for layers in self.lstm:
            layers.flatten_parameters()
            x_data, _  = layers(torch.unsqueeze(x[:, :, ii],2), (h0, c0))
            b, h, w = x_data.size()
#             x_data = x_data.view(b,-1)
            h_relu = torch.cat([h_relu, x_data], dim = 2)
            ii = ii+1
#         print(np.shape(h_relu))
        h_relu = F.relu(self.linear_1(h_relu[:, -1, :]))
        
        h_relu = self.linear1_1(h_relu)
        y_pred = self.linear_2(h_relu)
        
        return y_pred

In [19]:
for N in N1:
    for H in H1:
    #         H = 512
    #         H2 = 256
#         epochs  =1
        directory = '/raid/togzhan_syrymova/lump_project/models/'

        model = ConvLayerNet3(channels_in, channels_out, D_in, H, D_out, device, drop, kernel_size, stride).to(device)
        model_name = type(model).__name__

        model_name += '_pressure_' + 'lstm2_' + 'epochs_' \
        + str(epochs) + '_bs_' + str(N)+ '_H_' + str(H) \
        + '_drop_' + str(drop) \
        + '_ks_' + str(kernel_size) + '_stride_' + str(stride) + '_spect_1d'
        model_path = directory + model_name

        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        loss_fn = torch.nn.MSELoss()#BCELoss()  nn.MSELoss()
        print(model_name)
        print(model_path)
        print(model)

        # for H in H1:
        #     for N in N1:
        print(H)
        print(N)
        for e in range(epochs):
            total_loss = 0
            for i in range(0, train_x.shape[0], N):
                if i+N >= train_x.shape[0]:
                    x = train_x[i:]
                    y = train_y[i:]
                else:
                    x = train_x[i:i+N]
                    y = train_y[i:i+N]
                y_pred = model(x.to(device))
                loss = loss_fn(y_pred, y.to(device))
        #         loss_list[e] = loss.item()
                optimizer.zero_grad()
#                 loss.backward()
#                 optimizer.step()
                total_loss += loss.item()
            print("Epoch {}/{}, Train Loss: {:.3f}".format(e+1, epochs, total_loss))
            #pdb.set_trace()
            #Loss and accuracy on dev data
            with torch.no_grad():
                model.eval()
                output = model(dev_x.to(device))
                output = (output>0.5).float()
                acc    = accuracy_score(dev_y, output.cpu())
        #         accuracy_list[e] = acc
                if acc > dev_acc_max:
                    dev_acc_max=acc
                    epoch_max=e+1
                    torch.save(model, model_path)
              #correct = (output == dev_y.to(device)).float().sum()
    #             model.train()
                torch.save(model, model_path+'_last')
            print("Dev Accuracy: {:.3f}".format(acc))

        print("##############################################")
        print("Best dev accuracy is {:.3f} at epoch {}".format(dev_acc_max, epoch_max))
        print("Number of class 1 samples: ", (dev_y>0.5).float().sum().item())
        print("##############################################")

        ####################################################################################################
        #Evaluation
        ####################################################################################################
        with torch.no_grad():
        #             pdb.set_trace()
            model = torch.load(model_path)
            model.eval().to(device)
            output = model(test_x.to(device))
            output = (output>0.5).float()
            acc    = accuracy_score(test_y, output.cpu())
            print("Test Accuracy: {:.3f}".format(acc))
            print(model)

        with torch.no_grad():
        #             pdb.set_trace()
            model = torch.load(model_path+'_last')
            model.eval().to(device)
            output = model(test_x.to(device))
            output = (output>0.5).float()
            acc    = accuracy_score(test_y, output.cpu())
            print("Test Accuracy Last: {:.3f}".format(acc))
            print(model)
exit()

ConvLayerNet3_pressure_lstm2_epochs_100_bs_8_H_512_drop_0.3_ks_3_stride_2_spect_1d
/raid/togzhan_syrymova/lump_project/models/ConvLayerNet3_pressure_lstm2_epochs_100_bs_8_H_512_drop_0.3_ks_3_stride_2_spect_1d
ConvLayerNet3(
  (lstm): ModuleList(
    (0): LSTM(1, 32, batch_first=True)
    (1): LSTM(1, 32, batch_first=True)
    (2): LSTM(1, 32, batch_first=True)
    (3): LSTM(1, 32, batch_first=True)
    (4): LSTM(1, 32, batch_first=True)
    (5): LSTM(1, 32, batch_first=True)
    (6): LSTM(1, 32, batch_first=True)
    (7): LSTM(1, 32, batch_first=True)
    (8): LSTM(1, 32, batch_first=True)
    (9): LSTM(1, 32, batch_first=True)
    (10): LSTM(1, 32, batch_first=True)
    (11): LSTM(1, 32, batch_first=True)
    (12): LSTM(1, 32, batch_first=True)
    (13): LSTM(1, 32, batch_first=True)
    (14): LSTM(1, 32, batch_first=True)
  )
  (linear_1): Linear(in_features=480, out_features=512, bias=True)
  (linear1_1): Linear(in_features=512, out_features=64, bias=True)
  (linear_2): Linear(in_fe

KeyboardInterrupt: 

In [1]:
exit()
